# DPA攻击AES

## DPA攻击理论

* 在我们之前的文章中，已经发现了AES中S盒输出的汉明重量值会对能量轨迹产生影响，这已经足够我们进行对AES的攻击了。我们的目标是从从S盒的输出轨迹中分离出某一位。如果为`1`，则这一组的平均功率应该高于其他组。



    ![](img/Sbox_cpa_detail.png)

* S盒的输出来自子密钥和明文，由于正确的密钥和其他某个子密钥的平均值有差异，于是我们便有方法可以坚持子密钥是否正确，如果我们计算所有子密钥的均值差，将会看到正确的子密钥拥有最大的均值差。
* 步骤如下

    1. 捕获多组能量轨迹。
    2. 对S盒输出的最低位能轨进行统计分组。
    3. 计算均值差。
    4. 重复猜测子密钥。
    5. 最大的差值则为正确的子密钥。
    6. 对每个子密钥重复上述步骤。

In [ ]:
from tqdm import tnrange
import numpy as np
mean_diffs = np.zeros(255)
key_guess = []
known_key = known_keys[0]
plots = []
for subkey in tnrange(0, 16, desc="Attacking Subkey"):
    for kguess in tnrange(255, desc="Keyguess", leave=False):
        one_list = []
        zero_list = []
        
        for tnum in range(numtraces):
            if (intermediate(textin_array[tnum][subkey], kguess) & 1): #LSB is 1
                one_list.append(trace_array[tnum])
            else:
                zero_list.append(trace_array[tnum])
        one_avg = np.asarray(one_list).mean(axis=0)
        zero_avg = np.asarray(zero_list).mean(axis=0)
        mean_diffs[kguess] = np.max(abs(one_avg - zero_avg))
        if kguess == known_key[subkey]:
            plots.append(abs(one_avg - zero_avg))
    guess = np.argsort(mean_diffs)[-1]
    key_guess.append(guess)
    print(hex(guess) + "(real = 0x{:02X})".format(known_key[subkey]))
    #mean_diffs.sort()
    print(mean_diffs[guess])
    print(mean_diffs[known_key[subkey]])

## 幽灵峰问题

* 这是由于硬件设置导致的，我们不用对此过多考虑，详情见[论文](https://eprint.iacr.org/2005/311.pdf)。
* （Xenny）不过我在捕获的时候没遇到这个问题。